In [ ]:
import sys
sys.path.append(r"D:\data analysis\code\WBI_analysis")  # 例如 r"C:\Users\YourName\Project"
import AnalysisMethod as am
import pandas as pd
import os
import numpy as np
import WBIFunctions as WBI
import cv2
from sklearn.cluster import AgglomerativeClustering


In [ ]:
p_f_outside = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done'
f_folder = '20250115_4.5g-24d-ov_08'
p_f = p_f_outside+'\\'+f_folder

# (与荧光对时的)行为数据导入，不需要重新分析处理的
'''激光段的行为数据,帧率为30Hz左右'''
mot_mid_vol = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol' in f][0]
if 'pkl' in mot_mid_vol:
    df_mot_vol = pd.read_pickle(os.path.join(p_f, mot_mid_vol)).reset_index(drop=True)
else:
    df_mot_vol = pd.read_csv(os.path.join(p_f, mot_mid_vol))

calcium_intensity= np.load(os.path.join(p_f, 'calcium_intensity.npy'))
save_p = p_f.split('calcium_intensity.npy')[0]
print('文件大小:neuron*timestamp',calcium_intensity.shape)

In [ ]:
# 平滑calcium signal： 均值滤波器 (box filter)
scale = 1.5
for i in range(calcium_intensity.shape[0]):
    calcium_intensity[i] = (cv2.blur(calcium_intensity[i], (1, 7))*scale)[:,0]

In [ ]:
# 求前进速度和速率
df_mot_vol["head_velocity"] = df_mot_vol.apply(WBI.signed_norm, axis=1)
df_mot_vol["head_speed"] = df_mot_vol["head_velocity"].abs()
# 连续运动变量平滑
window_size = 15
# 计算移动平均值
df_mot_vol['sm_velocity'] = df_mot_vol['head_velocity'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_speed'] = df_mot_vol['head_speed'].rolling(window=window_size, min_periods=1).mean()
# # 平滑ctx
window_size = 15
# 计算移动平均值
df_mot_vol['sm_CTX'] = df_mot_vol['CTX_left'].rolling(window=window_size, min_periods=1).mean()
df_mot_vol['sm_ang'] = df_mot_vol['ang_velocity'].rolling(window=window_size, min_periods=1).mean()

# 定义图片保存文件夹路径
folder_path = os.path.join(p_f, 'AnalysisFigs')
# 检查文件夹是否存在
if not os.path.exists(folder_path):
    os.makedirs(folder_path)  # 创建文件夹
    print(f"文件夹 '{folder_path}' 已创建！")
else:
    print(f"文件夹 '{folder_path}' 已存在！")

In [ ]:
signal_save_path = folder_path+'\\HierClustering'
os.makedirs(signal_save_path, exist_ok=True)

In [ ]:
choose_index = 0
thresh = 5
links=['ward','average','average','complete']
affs=['euclidean','cosine','cityblock','cosine']
vmin=-0.5
vmax=1
smooth_kernel = 10
print('calcium_intensity.shape',calcium_intensity.shape)

# 使用 np.corrcoef(calcium_intensity) 计算神经元钙信号的相关性矩阵
# 聚类：使用 am.cluster 和指定的链接方式 (link=links[choose_index]) 
# 和距离度量 (aff=affs[choose_index]) 对相关性矩阵进行聚类。
idx=am.cluster(np.corrcoef(calcium_intensity),link=links[choose_index],aff=affs[choose_index])
# idx为聚类之后的索引
bound=np.cumsum(am.GetBound(np.corrcoef(calcium_intensity),link=links[choose_index],aff=affs[choose_index],
                            threshold=thresh).astype(int))
# 调用 am.GetBound 计算矩阵分区的边界（bound），通过 np.cumsum 累积求和获取完整的边界数组
print('边界', bound)

calcium_intensity = calcium_intensity[idx]
calcium_intensity_smd = calcium_intensity.copy()
# 平滑数据
if smooth_kernel:
    for i,k in enumerate(calcium_intensity_smd):
        calcium_intensity_smd[i] = cv2.blur(k,(1,smooth_kernel))[:,0]
# 计算相关性矩阵
w_p2m = np.corrcoef(calcium_intensity_smd)

# 聚类并将聚类的结果画在相关性矩阵旁边
link = links[choose_index]
aff = affs[choose_index]
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None,linkage=link, affinity=aff)
model = model.fit(w_p2m)
fig_h = int(10*round(calcium_intensity.shape[0]/10))
fig_w = int(0.7*round(calcium_intensity.shape[1]/100))
#     draw_calcium_curve(calcium_intensity,smooth_kernel=None,fig_size=(fig_w,fig_h),scale=1.5)

font_size = 100

'''输入事件开始index列表'''
neuron_ids = np.arange(calcium_intensity.shape[0])
col_draw = ['sm_velocity','sm_speed', 'sm_ang', 'sm_CTX','curvature', 'forward','turn_pc','turn_cor']

WBI.calcium_heatmap(calcium_intensity,df_mot_vol, col_draw,  neuron_ids,model,w_p2m, show_id_stride=10,
                show_vol_stride=500, heatmap_range=(0,0.6),wspace=0.06, hspace=0.2,bound_cluster=bound,
                unit_w=0.03, unit_h = 0.8, cal_height_ratio=30, smooth_kernel=smooth_kernel,
                font_size=font_size, font_color='black', idx = idx, vmin = vmin, vmax = vmax,
                    threshold=thresh, xlabel='Neuron Index',level=35, signal_save_path=signal_save_path)
'''注意这里是通过上一步排序后的钙信号数据！'''
calcium_intensity_sort = calcium_intensity.copy()

In [ ]:
block_path = signal_save_path+'\\BlockHighlight'
os.makedirs(block_path, exist_ok=True)
calcium_dict = {i: calcium_intensity_sort[i] for i in range(len(calcium_intensity_sort))}
WBI.block_visualize_traces(block_path,calcium_intensity, calcium_dict,df_mot_vol, bound, smooth=True, sigma=0.3,delta_y=1.2)

In [ ]:
calcium_intensity_sort_T = calcium_intensity_sort.T
# 创建列名
col_neuron_names = [f"neuron{i+1}" for i in range(calcium_intensity_sort_T.shape[1])]
df_calcium = pd.DataFrame(calcium_intensity_sort_T, columns=col_neuron_names)
# 按照行索引合并
df_cal_motion = pd.concat([df_calcium, df_mot_vol], axis=1)

In [ ]:
corr_folder = folder_path+'\\CorrAnalysis'
os.makedirs(corr_folder, exist_ok=True)
p_corr_ls = []  # 用于收集相关性分析的df的列表

In [ ]:
# 前后后退转换
# 开始
t_win = 25
neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
rev_start_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'start')
df_rev_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_start_idx, neu_cols,
                                           test_window = t_win, event_note='RevStart')  # 注意帧率为3左右
if df_rev_s_p is not None:
    df_rev_s_c = WBI.multi_compar(df_rev_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_rev_s_c)
    sel_n_s = df_rev_s_c[(df_rev_s_c['p_cor']<=0.05)]['Neuron']
    rev_folder = corr_folder+'\\Reverse'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_s_c, sel_n_s, rev_start_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='ReverseStart',x_label_str='Reverse Start',
                                       cmap='jet', unify_yaxis=True, save_path=rev_folder)


In [ ]:

# 结束
t_win = 20
rev_end_idx = WBI.get_event_start_idx(df_cal_motion, 'forward', 'end')
df_rev_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'forward', rev_end_idx,
                                            neu_cols,test_window = t_win, event_note='RevEnd')  # 注意帧率为3左右
if df_rev_e_p is not None:
    df_rev_e_c = WBI.multi_compar(df_rev_e_p,adj_method='fdr_bh')
    p_corr_ls.append(df_rev_e_c)
    sel_n_e = df_rev_e_c[(df_rev_e_c['p_cor']<=0.1)]['Neuron']

    rev_folder = corr_folder+'\\Reverse'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'forward',df_rev_e_c, sel_n_e, rev_end_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='ReverseEnd',x_label_str='Reverse End',
                                       cmap='jet', unify_yaxis=True, save_path=rev_folder)



In [ ]:
# omega turn(coiling开始)
# 开始
t_win = 10
# neu_cols = [col for col in df_cal_motion.columns if 'neuron' in col]
c_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_pc', 'start')
df_turn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_pc', c_start_idx,
                                             neu_cols,test_window = t_win, event_note='CoilingStart')  # 注意帧率为3左右
if df_turn_s_p is not None:
    df_turn_s_c = WBI.multi_compar(df_turn_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_turn_s_c)
    sel_n_turn = df_turn_s_c[(df_turn_s_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_pc',df_turn_s_c, sel_n_turn, c_start_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='CoilingStart',x_label_str='Coiling Start',
                                       cmap='jet', unify_yaxis=True, save_path=turn_folder)


In [ ]:

# omega turn (校正后的)
# 开始
t_win = 10
oturn_start_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'start')
df_oturn_s_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_start_idx,
                                              neu_cols,test_window = t_win, event_note='OmegaStart')  # 注意帧率为3左右
if df_oturn_s_p is not None:
    df_oturn_s_c = WBI.multi_compar(df_oturn_s_p,adj_method='fdr_bh')
    p_corr_ls.append(df_oturn_s_c)
    sel_n_oturn = df_oturn_s_c[(df_oturn_s_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_s_c, sel_n_oturn, oturn_start_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='OmegaTurnStart',x_label_str='OmegaTurn Start',
                                       cmap='jet', unify_yaxis=True, save_path=turn_folder)


In [ ]:
# omega turn end
# 开始
t_win = 10
oturn_end_idx = WBI.get_event_start_idx(df_cal_motion, 'turn_cor', 'end')
df_oturn_e_p = WBI.Paired_ttest_neu_discrete(df_cal_motion,'turn_cor', oturn_end_idx,
                                              neu_cols,test_window = t_win, event_note='OmegaEnd')  # 注意帧率为3左右
if df_oturn_e_p is not None:
    df_oturn_e_c = WBI.multi_compar(df_oturn_e_p,adj_method='fdr_bh')
    p_corr_ls.append(df_oturn_e_c)
    sel_n_oturn_e = df_oturn_e_c[(df_oturn_e_c['p_cor']<=0.05)]['Neuron']

    turn_folder = corr_folder+'\\OmegaTurn'
    WBI.plot_event_aligned_traces_combined(df_cal_motion,'turn_cor',df_oturn_e_c, sel_n_oturn_e, oturn_end_idx,
                                   pre_window = t_win,post_window=t_win, 
                                       sort_by='pre',sort_window=t_win,  frame_interval=0.3,
                                       heatmap_range = (0,0.6),save_title_str='OmegaTurnEnd',x_label_str='OmegaTurn End',
                                       cmap='jet', unify_yaxis=True, save_path=turn_folder)

In [ ]:
# 离散变量
beh_col = 'turn_pc'
p_uplim = 0.05
r_threshold = 0.25
df_neu_coil_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_coil_p['sign'] = 0
df_neu_coil_p.loc[(df_neu_coil_p['p_cor']<=p_uplim)&(df_neu_coil_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_coil_p)
# 显著的神经元
sign_neu_ls = df_neu_coil_p[(df_neu_coil_p['sign']==1)&((df_neu_coil_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward',"turn_pc"]
WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_coil_p,beh_col,
                               hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#DF6176",], num_bins=5,
                                 title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                               fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                               color='#2878B5', x_label='', y_label='',sigma = 5,
                               strip_color='#9AC9DB', label_pr=True, p_f=corr_folder+f'\\{beh_col}')

In [ ]:
beh_col = 'turn_cor'
p_uplim = 0.05
r_threshold = 0.25
df_neu_omega_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=[beh_col],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
df_neu_omega_p['sign'] = 0
df_neu_omega_p.loc[(df_neu_omega_p['p_cor']<=p_uplim)&(df_neu_omega_p['true_r'].abs()>r_threshold),'sign'] = 1
p_corr_ls.append(df_neu_omega_p)
# 显著的神经元
sign_neu_ls = df_neu_omega_p[(df_neu_omega_p['sign']==1)&((df_neu_omega_p['Event']==beh_col))]['Neuron'].unique()

hl_col_ls = ['forward','turn_pc',"turn_cor"]
WBI.plot_lines_neural_activity_disc(df_cal_motion, sign_neu_ls,df_neu_omega_p,beh_col=beh_col,
                               hl_col =hl_col_ls,hl_color=[ "#1BA4FF","#F0EC61","#DF6176",], num_bins=5,
                                 title=f"NeuCorrWith{beh_col.replace('sm_','')}HighlightedBy{'-'.join(hl_col_ls)}", color_map='inferno',
                               fs=25, x_bin=0.2, time_range_sel=[],axis_invisible = True,
                               color='#2878B5', x_label='', y_label='',sigma = 5,
                               strip_color='#9AC9DB', label_pr=True, p_f=corr_folder+f'\\{beh_col}')

In [ ]:
# 连续变量
df_neu_con_p = WBI.corr_neu_con_var(df_cal_motion,behavior_col_ls=['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX'],
                     neuron_col_str='neuron',adj_method = 'fdr_bh' )
p_corr_ls.append(df_neu_con_p)
r_threhold = 0.2
x_ticklabels=[f"{i}" for i in range(1, 1+len(df_neu_con_p['Neuron'].unique()))]
WBI.plot_correlation_heatmap_nomask_from_df(df_neu_con_p,
                                 row_col = 'Event',
                                 col_col='Neuron',
                                     alpha_original=0.05,
                                     title="Neuron-Behavior Correlation Heatmap",
                                 mask_color='#f0f0f0',   # 灰白
                                 x_ticklabels=x_ticklabels,
                                 r_threshold=r_threhold,
                                 save_path=corr_folder,
                                 save_title='CorrNeuAllContinous',
                                    fs = 12.5)
beh_sig_ls = ['sm_velocity','sm_speed','sm_ang','curvature','sm_CTX']  # 可视化行为列表
# beh_sig_ls = ['sm_ang','curvature','sm_CTX']
for beh_col in beh_sig_ls:
    WBI.visualize_cont_beh_by_neu(df_neu_con_p,df_cal_motion, beh_col, p_uplim=0.05, r_threshold=0.2, p_f_con = corr_folder)    

In [ ]:
# 写出p值文件
df_corr_p = pd.concat(p_corr_ls, axis=0, ignore_index=True)
df_corr_p['File'] = f_folder
corr_p_path = corr_folder+f'\\{f_folder}_corr_p_cor.csv'
df_corr_p.to_csv(corr_p_path, index=False)